In [1]:
!pip uninstall -y numpy
!pip install numpy==1.26.4
!pip install scikit-surprise


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2469539 sha256=926bab28bcbffa66adf270591c1fbfc564d8a455cb74d1c8be9640b2d76e02ce
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [2]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

print("🎉 All good! You're ready to go.")


🎉 All good! You're ready to go.


In [5]:
# Step 3: Load your dataset
data = pd.read_csv('/content/intern_learning_history.csv')

# Step 4: Prepare the data for Surprise library
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(data[['intern_id', 'course_id', 'rating']], reader)

# Step 5: Train-test split
trainset, testset = train_test_split(surprise_data, test_size=0.2)

# Step 6: Build model using SVD (Matrix Factorization)
model = SVD()
model.fit(trainset)

# Step 7: Predict on test set
predictions = model.test(testset)

# Step 8: Evaluate
accuracy.rmse(predictions)

# Step 9: Recommend for a specific intern
def get_recommendations(intern_id, course_ids, top_n=3):
    # Predict ratings for all unseen courses
    unseen = [cid for cid in course_ids if not ((data['intern_id'] == intern_id) & (data['course_id'] == cid)).any()]
    intern_predictions = [(cid, model.predict(intern_id, cid).est) for cid in unseen]
    intern_predictions.sort(key=lambda x: x[1], reverse=True)
    return intern_predictions[:top_n]

# Example: Recommend for intern_id = 1
all_courses = pd.read_csv('/content/course_metadata.csv')['course_id'].tolist()
recommendations = get_recommendations(1, all_courses)
print("Recommended courses for Intern 1:")
for cid, score in recommendations:
    print(f"{cid} - Predicted Rating: {round(score, 2)}")

RMSE: 1.9923
Recommended courses for Intern 1:
C103 - Predicted Rating: 4.53
C104 - Predicted Rating: 4.5
C105 - Predicted Rating: 4.5
